# COVID-19 Prediction Analysis

In [1]:
# Suppress warning messages
import warnings 
warnings.filterwarnings('ignore')

In [2]:
# Import general dependencies
import os 
import pandas as pd
from pprint import pprint
from pathlib import Path
import matplotlib.pyplot as plt
# from collections import Counter
import numpy as np
import datetime

In [3]:
#Import connection dependencies
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
import psycopg2

In [4]:
# Import machine learning dependencies 
import sklearn as skl
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
# from sklearn.metrics import accuracy_score, confusion_matrix
# from imblearn.metrics import classificationn_report_imbalanced 
from sklearn.metrics import mean_squared_error

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint

# import keras_tuner as kt

### Data Source

**Our World in Data COVID-19 Dataset Citation**
Hannah Ritchie, Edouard Mathieu, Lucas Rodés-Guirao, Cameron Appel, Charlie Giattino, Esteban Ortiz-Ospina, Joe Hasell, Bobbie Macdonald, Diana Beltekian and Max Roser (2020) - "Coronavirus Pandemic (COVID-19)". Published online at OurWorldInData.org. Retrieved from: [Our World In Data / Coronavirus](https://ourworldindata.org/coronavirus)

In [5]:
# Create connection to SQL database
conn = psycopg2.connect(
    host="finalproject.cjgqtjwgyywe.us-east-2.rds.amazonaws.com",
    database="postgres",
    user="postgres",
    password="FinalProject1")

In [6]:
# Read the OneWorldInData COVID-19 database into a DataFrame
df = pd.read_sql("SELECT * from all_countries_data", conn)
print(df.shape)
df.head()

(187000, 68)


,id_row,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,89248,ITA,Europe,Italy,2020-09-09,281583.0,1430.0,1438.286,35577.0,14.0,...,19.8,27.8,NaN,3.18,83.51,0.892,NaN,NaN,NaN,NaN
1,187610,USA,North America,United States,2020-10-08,7611057.0,57890.0,46397.714,211879.0,939.0,...,19.1,24.6,NaN,2.77,78.86,0.926,NaN,NaN,NaN,NaN
2,187638,USA,North America,United States,2020-11-05,9670209.0,128081.0,99990.857,235450.0,1167.0,...,19.1,24.6,NaN,2.77,78.86,0.926,NaN,NaN,NaN,NaN
3,187660,USA,North America,United States,2020-11-27,13164517.0,214634.0,170510.571,266104.0,1622.0,...,19.1,24.6,NaN,2.77,78.86,0.926,NaN,NaN,NaN,NaN
4,187677,USA,North America,United States,2020-12-14,16618535.0,209825.0,220695.143,304248.0,1655.0,...,19.1,24.6,NaN,2.77,78.86,0.926,NaN,NaN,NaN,NaN


In [7]:
# Set the id_row column as index and reorder records by id.
df = df.set_index('id_row').sort_values(by='id_row')
print(df.shape)
df.head()

(187000, 67)


,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
id_row,,,,,,,,,,,,,,,,,,,,,
0,AFG,Asia,Afghanistan,2020-02-24,5.0,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-02-25,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-02-26,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-02-27,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-02-28,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN


In [8]:
# Check non-null counts and data types.
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 187000 entries, 0 to 198845
Data columns (total 67 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   iso_code                                    187000 non-null  object 
 1   continent                                   187000 non-null  object 
 2   location                                    187000 non-null  object 
 3   date                                        187000 non-null  object 
 4   total_cases                                 179392 non-null  float64
 5   new_cases                                   179111 non-null  float64
 6   new_cases_smoothed                          177999 non-null  float64
 7   total_deaths                                160966 non-null  float64
 8   new_deaths                                  160731 non-null  float64
 9   new_deaths_smoothed                         159637 non-null  float64
 

In [9]:
# Count the records in the COVID-19 DataFrame.
records = df["location"].count()
print(f"There are {records} location-specific records in the OWID COVID-19 dataset as of July 5th, 2022.")

There are 187000 location-specific records in the OWID COVID-19 dataset as of July 5th, 2022.


In [10]:
# Check the earliest and latest date in the dataset. 
df["date"].sort_values()

id_row
6822      2020-01-01
115431    2020-01-01
6823      2020-01-02
115432    2020-01-02
115433    2020-01-03
             ...    
18933     2022-07-04
50735     2022-07-04
121987    2022-07-04
49879     2022-07-04
198845    2022-07-04
Name: date, Length: 187000, dtype: object

The OWID COVID-19 data entry started on January 1st, 2020 and has been daily updated till July 4th, 2022 when we pulled the dataset from the [OWID/COVID-19-data GitHub Repository](https://github.com/owid/covid-19-data/tree/master/public/data).

In [11]:
# Count the unique locations included in the COVID-19 dataset. 
countries_count = len(df["location"].unique())
print(f"There are {countries_count} unique values in the OWID COVID-19 location column.")

There are 230 unique values in the OWID COVID-19 location column.


In [12]:
# List the locations included in the COVID-19 dataset.
locations_list = df["location"].unique()
print(f"Locations list: {locations_list}")

Locations list: ['Afghanistan' 'Albania' 'Algeria' 'Andorra' 'Angola' 'Anguilla'
 'Antigua and Barbuda' 'Argentina' 'Armenia' 'Aruba' 'Australia' 'Austria'
 'Azerbaijan' 'Bahamas' 'Bahrain' 'Bangladesh' 'Barbados' 'Belarus'
 'Belgium' 'Belize' 'Benin' 'Bermuda' 'Bhutan' 'Bolivia'
 'Bonaire Sint Eustatius and Saba' 'Bosnia and Herzegovina' 'Botswana'
 'Brazil' 'British Virgin Islands' 'Brunei' 'Bulgaria' 'Burkina Faso'
 'Burundi' 'Cambodia' 'Cameroon' 'Canada' 'Cape Verde' 'Cayman Islands'
 'Central African Republic' 'Chad' 'Chile' 'China' 'Colombia' 'Comoros'
 'Congo' 'Cook Islands' 'Costa Rica' "Cote d'Ivoire" 'Croatia' 'Cuba'
 'Curacao' 'Cyprus' 'Czechia' 'Democratic Republic of Congo' 'Denmark'
 'Djibouti' 'Dominica' 'Dominican Republic' 'Ecuador' 'Egypt'
 'El Salvador' 'Equatorial Guinea' 'Eritrea' 'Estonia' 'Eswatini'
 'Ethiopia' 'Faeroe Islands' 'Falkland Islands' 'Fiji' 'Finland' 'France'
 'French Polynesia' 'Gabon' 'Gambia' 'Georgia' 'Germany' 'Ghana'
 'Gibraltar' 'Greece' 'Gre

## Data Manipulation

There are 2 intended purposes of the prediction models: 
1. Predicting the daily number of new COVID-19 cases per 100,000 people in all included countries/regions.
2. Predicting the daily number of new COVID-19-related deaths per 1,000 people in all included countries/regions. 

#### Removing Irrelevant Variables

In [13]:
# Drop all columns related to COVID-19 testing. 
tests_drop = ['total_tests', 'new_tests', 'total_tests_per_thousand', 
             'new_tests_per_thousand', 'new_tests_smoothed', 
             'new_tests_smoothed_per_thousand', 'positive_rate', 
             'tests_per_case', 'tests_units']
no_test_df = df.drop(columns=tests_drop, axis=1)
print(no_test_df.shape)

(187000, 58)


In [14]:
# Drop all columns related to calculated excess_mortality.
mortality_drop = ['excess_mortality', 'excess_mortality_cumulative',
                  'excess_mortality_cumulative_absolute', 'excess_mortality_cumulative_per_million']
no_mortality_df = no_test_df.drop(columns=mortality_drop, axis=1)
print(no_mortality_df.shape)        

(187000, 54)


In [15]:
# Drop all columns related to icu reporting. 
icu_drop = ['icu_patients', 'icu_patients_per_million', 'hosp_patients', 
            'hosp_patients_per_million', 'weekly_icu_admissions', 
            'weekly_icu_admissions_per_million', 'weekly_hosp_admissions', 
            'weekly_hosp_admissions_per_million']
no_icu_df = no_mortality_df.drop(columns=icu_drop, axis=1)
print(no_icu_df.shape)  

(187000, 46)


In [16]:
# Drop unnecessary general columns. 
general_drop = ['iso_code', 'continent', 'aged_70_older']
general_df = no_icu_df.drop(columns=general_drop, axis=1)
print(general_df.shape)

(187000, 43)


#### Handling Missing Values of Main Features in All Locations

In [17]:
# Check the count of missing values in total_cases in all locations.
general_df.total_cases.isnull().sum()

7608

In [18]:
# Drop missing values from 'total_cases'.
total_cases_general_df = general_df.dropna(subset =["total_cases"], how="all")
print(total_cases_general_df.shape)
print(f" There are {total_cases_general_df['total_cases'].isnull().sum()} missing values in the total cases.")

(179392, 43)
 There are 0 missing values in the total cases.


In [19]:
# Check the count of missing values in new cases 
total_cases_general_df.new_cases.isnull().sum()

281

In [20]:
# Fill null values in new_cases with (0)
cases_general_df = total_cases_general_df.copy()
cases_general_df['new_cases'] = cases_general_df['new_cases'].fillna(0)
print(cases_general_df.shape)
cases_general_df['new_cases'].isnull().sum()

(179392, 43)


0

In [21]:
cases_general_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 179392 entries, 0 to 198845
Data columns (total 43 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   location                                    179392 non-null  object 
 1   date                                        179392 non-null  object 
 2   total_cases                                 179392 non-null  float64
 3   new_cases                                   179392 non-null  float64
 4   new_cases_smoothed                          177999 non-null  float64
 5   total_deaths                                160965 non-null  float64
 6   new_deaths                                  160730 non-null  float64
 7   new_deaths_smoothed                         159637 non-null  float64
 8   total_cases_per_million                     179392 non-null  float64
 9   new_cases_per_million                       179111 non-null  float64
 

In [ ]:
# # Drop rows missing new_cases_smoothed values.
# avg_cases_general_df = cases_general_df.copy()
# avg_cases_general_df = avg_cases_general_df.dropna(subset =["new_cases_smoothed"], how="all")
# print(avg_cases_general_df.shape)
# print(f" There are {avg_cases_general_df['new_cases_smoothed'].isnull().sum()} missing values in the 7-day average of new COVID-19 cases.")

To predict the daily number of new COVID-19 cases, we will focus on processing the following columns and creating new features off of these: (**total_cases, new_cases, total_cases_per_million, new_cases_per_million, reproduction_rate, total_vaccinations, people_fully_vaccinated, total_vaccinations_per_hundred, people_fully_vaccinated_per_hundred, stringency_index, population, population_density, median_age, aged_65_older, gdp_per_capita, cardiovasc_death_rate, diabetes_prevalence, life_expectancy, human_development_index**). 

We will not be using smoothed numbers for now, but we will leave these columns in a separate dataframe that contains averages in case we were able to revisit the code and run a more elaborate feature engineering including calculating rolling averages of new vaccines and people_fully_vaccinated. At that point, we could use the cleaned dataframe and handle the missing values of the smoothed columns: (**new_cases_smoothed, new_cases_smoothed_per_million, new_vaccinations_smoothed, new_vaccinations_smoothed_per_million**).

The reason we will leave **female_smokers** and **male_smokers** out of the new_cases prediction features for now is the substantial number of missing values. 
*Reconsider inclusing **female_smokers**, and **male_smokers** in the features for new_cases prediction model if the model needs feature enhancement. The logic behind it is that smokers might be more symptomatic when infected with COVID-19, hence they seek testing and that would increase the documented number of the cases. Same concept applies to the populations' rates of cardiovascular_death and diabetes_prevalence. 

We will handle all deaths-related columns (**total_deaths, new_deaths, new_deaths_smoothed, total_deaths_per_million, new_deaths_per_million, new_deaths_smoothed_per_million**), in addition to other outcome columns such as **female_smokers**, **male_smokers**, and **hospital_beds_per_thousand** later in the deaths prediction stage where we will be applying the processing on a deaths DataFrame.

Reproduction Rate: is a way of measuring an infectious disease’s capacity to spread. The R number signifies the average number of people that one infected person will pass the virus to. <br>
In the OWID COVID-19 dataset, the column **reproduction_rate**: represents a real-time estimate of the effective reproduction rate (R) of COVID-19.<br>
We will drop missing values of the reproduction rate since replacing them with zero might skew the data and influence the results. Furthermore, there was no clear pattern that explains why these values were missing from the dataset when it comes to location or total_cases values. 

In [28]:
# Check the count of missing values in reproduction rate column.
cases_general_df.reproduction_rate.isnull().sum()

31175

In [29]:
# Drop rows missing reproduction rate values.
r_general_df = cases_general_df.copy()
r_general_df = r_general_df.dropna(subset =["reproduction_rate"], how="all")
print(r_general_df.shape)
print(f" There are {r_general_df['reproduction_rate'].isnull().sum()} missing values in reproduction rate of COVID-19.")

(148217, 43)
 There are 0 missing values in reproduction rate of COVID-19.


In [36]:
# Check the location counts after dropping empty reproduction rate values.
r_general_df.location.nunique()

190

In [27]:
r_general_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 148217 entries, 34 to 198843
Data columns (total 43 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   location                                    148217 non-null  object 
 1   date                                        148217 non-null  object 
 2   total_cases                                 148217 non-null  float64
 3   new_cases                                   148217 non-null  float64
 4   new_cases_smoothed                          148026 non-null  float64
 5   total_deaths                                146068 non-null  float64
 6   new_deaths                                  145872 non-null  float64
 7   new_deaths_smoothed                         145374 non-null  float64
 8   total_cases_per_million                     148217 non-null  float64
 9   new_cases_per_million                       148059 non-null  float64


#### Vaccinations Transformation

OWID notes:
- Vaccinations data were updated daily for 218 locations of the original dataset out of 244.
- On 6 January 2021, we added two variables for daily vaccinations to our complete dataset.
- On 26 January 2021, we added 4 variables on people vaccinated and people fully vaccinated to our complete dataset.
- On 11 August 2021, we added the metric total_boosters to our vaccination data. This counts the total number of booster doses (doses administered beyond the number prescribed by the vaccination protocol).
- On 15 November 2021, we added the metrics new_people_vaccinated_smoothed and new_people_vaccinated_smoothed_per_hundred to our vaccination data. They count the daily number of people receiving their first vaccine dose.
- Since 29 March 2022, vaccination data is no longer updated on a daily basis. Updates now are only on weekdays (Monday until Friday).

We will fill the vaccinations nulls with zeros because the absence of vaccines emphasizes certain trends in the pandemic; as such, vaccines initiation is expected to decrease the number of cases and covid-related deaths. 

###### We presume that vaccination  has played an integral role in the COVID-19 pandemic trajectory, and its influence may be observed over both COVID-19 cases and deaths.

In [33]:
# Check the date range of vaccinations records
vac_dates = r_general_df[r_general_df.total_vaccinations.notnull()]
vac_dates.date.sort_values()

id_row
134855    2020-12-02
98744     2020-12-04
98747     2020-12-07
47632     2020-12-08
134861    2020-12-08
             ...    
63678     2022-07-02
67955     2022-07-02
70514     2022-07-02
83000     2022-07-02
198843    2022-07-02
Name: date, Length: 44719, dtype: object

In [30]:
# Check the count of total_vaccinations records in the whole date range of the current dataset
r_general_df.total_vaccinations.notnull().sum()

44719

**There are 44,679 records of vaccinations from all locations.**

In [31]:
# Check the count of missing values in total_vaccinations column
print(r_general_df.total_vaccinations.isnull().sum())
print(r_general_df.total_vaccinations_per_hundred.isnull().sum())

103498
103498


In [34]:
# Fill null values in total_vaccinations and total_vaccinations_per_hundred with (0)
total_vac_general_df = r_general_df.copy()
total_vac_general_df['total_vaccinations'] = total_vac_general_df['total_vaccinations'].fillna(0)
total_vac_general_df['total_vaccinations_per_hundred'] = total_vac_general_df['total_vaccinations_per_hundred'].fillna(0)
print(total_vac_general_df.shape)
total_vac_general_df[['total_vaccinations', 'total_vaccinations_per_hundred']].isnull().sum()

(148217, 43)


total_vaccinations                0
total_vaccinations_per_hundred    0
dtype: int64

In [37]:
# Check the count of missing values in people_fully_vaccinated column
print(total_vac_general_df.people_fully_vaccinated.isnull().sum())
print(total_vac_general_df.people_fully_vaccinated_per_hundred.isnull().sum())

108105
108105


In [38]:
# Fill null values in people_fully_vaccinated and people_fully_vaccinated_per_hundred with (0)
ppl_vac_general_df = total_vac_general_df.copy()
ppl_vac_general_df['people_fully_vaccinated'] = ppl_vac_general_df['people_fully_vaccinated'].fillna(0)
ppl_vac_general_df['people_fully_vaccinated_per_hundred'] = ppl_vac_general_df['people_fully_vaccinated_per_hundred'].fillna(0)
print(ppl_vac_general_df.shape)
ppl_vac_general_df[['people_fully_vaccinated', 'people_fully_vaccinated_per_hundred']].isnull().sum()

(148217, 43)


people_fully_vaccinated                0
people_fully_vaccinated_per_hundred    0
dtype: int64

In [39]:
# Convert the 'date' column to a datetime.
ppl_vac_general_df['date'] = ppl_vac_general_df['date'].apply(pd.to_datetime)
print(ppl_vac_general_df.date.dtype)

datetime64[ns]


In [41]:
ppl_vac_general_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 148217 entries, 34 to 198843
Data columns (total 43 columns):
 #   Column                                      Non-Null Count   Dtype         
---  ------                                      --------------   -----         
 0   location                                    148217 non-null  object        
 1   date                                        148217 non-null  datetime64[ns]
 2   total_cases                                 148217 non-null  float64       
 3   new_cases                                   148217 non-null  float64       
 4   new_cases_smoothed                          148026 non-null  float64       
 5   total_deaths                                146068 non-null  float64       
 6   new_deaths                                  145872 non-null  float64       
 7   new_deaths_smoothed                         145374 non-null  float64       
 8   total_cases_per_million                     148217 non-null  float64     

In [40]:
# Create a DataFrame before March 29, 2022
mar_general_df = ppl_vac_general_df.copy()
mar_general_df = mar_general_df.loc[mar_general_df['date'] < '2022-03-29']
print(mar_general_df.shape)
mar_general_df.head()

(129977, 43)


,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
id_row,,,,,,,,,,,,,,,,,,,,,
34,Afghanistan,2020-03-29,114.0,8.0,11.429,4.0,2.0,0.571,2.862,0.201,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
35,Afghanistan,2020-03-30,114.0,0.0,10.571,4.0,0.0,0.429,2.862,0.000,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
36,Afghanistan,2020-03-31,166.0,52.0,17.714,4.0,0.0,0.429,4.167,1.305,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
37,Afghanistan,2020-04-01,192.0,26.0,16.857,4.0,0.0,0.429,4.820,0.653,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
38,Afghanistan,2020-04-02,235.0,43.0,22.143,4.0,0.0,0.286,5.899,1.079,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511


In [42]:
mar_general_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 129977 entries, 34 to 198747
Data columns (total 43 columns):
 #   Column                                      Non-Null Count   Dtype         
---  ------                                      --------------   -----         
 0   location                                    129977 non-null  object        
 1   date                                        129977 non-null  datetime64[ns]
 2   total_cases                                 129977 non-null  float64       
 3   new_cases                                   129977 non-null  float64       
 4   new_cases_smoothed                          129859 non-null  float64       
 5   total_deaths                                127831 non-null  float64       
 6   new_deaths                                  127695 non-null  float64       
 7   new_deaths_smoothed                         127208 non-null  float64       
 8   total_cases_per_million                     129977 non-null  float64     

In [43]:
vac_drop = ['people_vaccinated', 'total_boosters', 'new_vaccinations', 
            'new_vaccinations_smoothed', 'people_vaccinated_per_hundred', 
            'total_boosters_per_hundred', 'new_vaccinations_smoothed_per_million',
           'new_people_vaccinated_smoothed', 'new_people_vaccinated_smoothed_per_hundred']
vac_general_df = mar_general_df.drop(columns=vac_drop, axis=1)
print(vac_general_df.shape)  

(129977, 34)


**stringency_index**: Government Response Stringency Index is a composite measure based on 9 response indicators including school closures, workplace closures, and travel bans, rescaled to a value from 0 to 100 (100 = strictest response)

In [45]:
# Check the count of missing values in the stringency_index column
vac_general_df.stringency_index.isnull().sum()

7942

In [47]:
# Fill null values in stringency_index with (0)
str_general_df = vac_general_df.copy()
str_general_df['stringency_index'] = str_general_df['stringency_index'].fillna(0)
print(str_general_df.shape)
str_general_df['stringency_index'].isnull().sum()

(129977, 34)


0

**population_density**: Number of people divided by land area, measured in square kilometers, most recent year available.

In [48]:
# Check the count of missing values in the population_density column
str_general_df.population_density.isnull().sum()

2101

In [49]:
# Drop rows missing population density values.
pop_general_df = str_general_df.copy()
pop_general_df = pop_general_df.dropna(subset =["population_density"], how="all")
print(pop_general_df.shape)
print(f" There are {pop_general_df['population_density'].isnull().sum()} missing values in population_density of COVID-19 dataset.")

(127876, 34)
 There are 0 missing values in population_density of COVID-19 dataset.


In [50]:
pop_general_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 127876 entries, 34 to 198747
Data columns (total 34 columns):
 #   Column                               Non-Null Count   Dtype         
---  ------                               --------------   -----         
 0   location                             127876 non-null  object        
 1   date                                 127876 non-null  datetime64[ns]
 2   total_cases                          127876 non-null  float64       
 3   new_cases                            127876 non-null  float64       
 4   new_cases_smoothed                   127760 non-null  float64       
 5   total_deaths                         125736 non-null  float64       
 6   new_deaths                           125600 non-null  float64       
 7   new_deaths_smoothed                  125118 non-null  float64       
 8   total_cases_per_million              127876 non-null  float64       
 9   new_cases_per_million                127790 non-null  float64       


In [51]:
# Check the count of missing values in median_age
pop_general_df.median_age.isnull().sum()

4238

In [52]:
# Drop rows missing median age values.
med_age_general_df = pop_general_df.copy()
med_age_general_df = med_age_general_df.dropna(subset =["median_age"], how="all")
print(med_age_general_df.shape)
print(f" There are {med_age_general_df['median_age'].isnull().sum()} missing values in median_age of COVID-19 dataset.")

(123638, 34)
 There are 0 missing values in median_age of COVID-19 dataset.


In [53]:
# Check the count of missing values in aged_65_older 
med_age_general_df.isnull().sum()

location                                   0
date                                       0
total_cases                                0
new_cases                                  0
new_cases_smoothed                       114
total_deaths                            1885
new_deaths                              2019
new_deaths_smoothed                     2482
total_cases_per_million                    0
new_cases_per_million                     84
new_cases_smoothed_per_million           114
total_deaths_per_million                1885
new_deaths_per_million                  2019
new_deaths_smoothed_per_million         2482
reproduction_rate                          0
total_vaccinations                         0
people_fully_vaccinated                    0
total_vaccinations_per_hundred             0
people_fully_vaccinated_per_hundred        0
stringency_index                           0
population                                 0
population_density                         0
median_age

In [ ]:
# # Drop rows missing aged 65 older values.
# older_general_df = med_age_general_df.copy()
# older_general_df = older_general_df.dropna(subset =["aged_65_older"], how="all")
# print(older_general_df.shape)
# print(f" There are {older_general_df['aged_65_older'].isnull().sum()} missing values in aged_65_older of COVID-19 dataset.")

In [55]:
# Drop rows missing gdp_per_capita values.
gdp_general_df = med_age_general_df.copy()
gdp_general_df = gdp_general_df.dropna(subset =["gdp_per_capita"], how="all")
print(gdp_general_df.shape)
print(f" There are {gdp_general_df['gdp_per_capita'].isnull().sum()} missing values in gdp_per_capita of COVID-19 dataset.")

(122198, 34)
 There are 0 missing values in gdp_per_capita of COVID-19 dataset.


In [65]:
gdp_general_df.isnull().sum()

location                                   0
date                                       0
total_cases                                0
new_cases                                  0
new_cases_smoothed                       114
total_deaths                            1885
new_deaths                              2017
new_deaths_smoothed                     2480
total_cases_per_million                    0
new_cases_per_million                     84
new_cases_smoothed_per_million           114
total_deaths_per_million                1885
new_deaths_per_million                  2017
new_deaths_smoothed_per_million         2480
reproduction_rate                          0
total_vaccinations                         0
people_fully_vaccinated                    0
total_vaccinations_per_hundred             0
people_fully_vaccinated_per_hundred        0
stringency_index                           0
population                                 0
population_density                         0
median_age

In [57]:
#Drop column extreme_poverty
no_poverty_general_df = gdp_general_df.copy()
no_poverty_general_df = no_poverty_general_df.drop(columns=["extreme_poverty"], axis=1)
print(no_poverty_general_df.shape)  

(122198, 33)


In [64]:
no_poverty_general_df.isnull().sum()

location                                   0
date                                       0
total_cases                                0
new_cases                                  0
new_cases_smoothed                       114
total_deaths                            1885
new_deaths                              2017
new_deaths_smoothed                     2480
total_cases_per_million                    0
new_cases_per_million                     84
new_cases_smoothed_per_million           114
total_deaths_per_million                1885
new_deaths_per_million                  2017
new_deaths_smoothed_per_million         2480
reproduction_rate                          0
total_vaccinations                         0
people_fully_vaccinated                    0
total_vaccinations_per_hundred             0
people_fully_vaccinated_per_hundred        0
stringency_index                           0
population                                 0
population_density                         0
median_age

In [59]:
# Drop column handwashing_facilities ( we lose more data than plausible) instead of filling with zeros becuase it will skew the data
# The public health status of the location should be reflected in other columns that we retained
hand_general_df = no_poverty_general_df.copy()
hand_general_df = hand_general_df.drop(columns=["handwashing_facilities"], axis=1)
print(hand_general_df.shape)

(122198, 32)


In [ ]:
# # Drop rows missing life_expectancy values.
# life_exp_general_df = hand_general_df.copy()
# # life_exp_general_df = life_exp_general_df.dropna(subset =["life_expectancy"], how="all")
# print(life_exp_general_df.shape)
# print(f" There are {life_exp_general_df['life_expectancy'].isnull().sum()} missing values in life_expectancy of COVID-19 dataset.")

In [63]:
hand_general_df.isnull().sum()

location                                   0
date                                       0
total_cases                                0
new_cases                                  0
new_cases_smoothed                       114
total_deaths                            1885
new_deaths                              2017
new_deaths_smoothed                     2480
total_cases_per_million                    0
new_cases_per_million                     84
new_cases_smoothed_per_million           114
total_deaths_per_million                1885
new_deaths_per_million                  2017
new_deaths_smoothed_per_million         2480
reproduction_rate                          0
total_vaccinations                         0
people_fully_vaccinated                    0
total_vaccinations_per_hundred             0
people_fully_vaccinated_per_hundred        0
stringency_index                           0
population                                 0
population_density                         0
median_age

In [62]:
# # Drop rows missing human_development_index values.
# hum_dev_general_df = hand_general_df.copy()
# hum_dev_general_df = hum_dev_general_df.dropna(subset =["human_development_index"], how="all")
# print(hum_dev_general_df.shape)
# print(f" There are {hum_dev_general_df['human_development_index'].isnull().sum()} missing values in human_development_index of COVID-19 dataset.")

(122198, 32)
 There are 0 missing values in human_development_index of COVID-19 dataset.


In [67]:
hum_dev_general_df = hand_general_df.copy()
hum_dev_general_df.shape

(122198, 32)

## Feature Engineering

1. Replace the date column with covid_days that counts the number of days into the pandemic.
2. Create a new column to calculate daily difference in total_vaccinations.
3. Create a new column to calculate daily difference in people_fully_vaccinated.
4. Normalize: 
    - total_cases_per_100K
    - new_cases_per_100K
    - total_vaccinations_per_100K
    - people_fully_vaccinated_per_100K
    - daily_vaccines_per_100K
    - daily_people_fully_vaccinated_per_100K

#### Create covid_days

The date of reporting carries a significant importance in the dataset, because it conveys the state of the COVID-19 pandemic in each location in comparison to other locations at the same timescale of the health crisis. It may also highlight chronoligcal relations between other variables in the dataset. 

The first announcement of COVID-19 infections was on December 31, 2019. <br>
The World Health Organization China Country Office is informed of a number cases of pneumonia of unknown etiology (unknown cause) detected in Wuhan, Hubei Province.

The OWID COVID-19 data entry started on January 1st, 2020 and has been daily updated till July 4th, 2022 when we pulled the dataset from the [OWID/COVID-19-data GitHub Repository](https://github.com/owid/covid-19-data/tree/master/public/data).

Although COVID-19 infection did not appear in all locations of the world at the same time, it is important to compare the pandemic statue across locations at the same timescale. <br>

We will calculate the days passed between the first reporting of COVID-19 in the dataset, which is January 1st, 2020 and the date of the each record for each location. 

In [68]:
# Check the earliest and latest date in the dataset. 
hum_dev_general_df["date"].sort_values()

id_row
36732    2020-01-23
36733    2020-01-24
36734    2020-01-25
36735    2020-01-26
36736    2020-01-27
            ...    
173172   2022-03-28
68702    2022-03-28
70418    2022-03-28
57451    2022-03-28
198747   2022-03-28
Name: date, Length: 122198, dtype: datetime64[ns]

In [69]:
# Create a new column for days of COVID-19 and calculate its value from Jan 1, 2020.
date_to_convert = '2020-01-01'
start_date = pd.to_datetime(date_to_convert, exact=True)
hum_dev_general_df['covid_days'] = (hum_dev_general_df['date'] - start_date).dt.days
print(hum_dev_general_df.shape)
hum_dev_general_df.head(10)

(122198, 33)


,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,...,aged_65_older,gdp_per_capita,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,hospital_beds_per_thousand,life_expectancy,human_development_index,covid_days
id_row,,,,,,,,,,,,,,,,,,,,,
34,Afghanistan,2020-03-29,114.0,8.0,11.429,4.0,2.0,0.571,2.862,0.201,...,2.581,1803.987,597.029,9.59,NaN,NaN,0.5,64.83,0.511,88
35,Afghanistan,2020-03-30,114.0,0.0,10.571,4.0,0.0,0.429,2.862,0.000,...,2.581,1803.987,597.029,9.59,NaN,NaN,0.5,64.83,0.511,89
36,Afghanistan,2020-03-31,166.0,52.0,17.714,4.0,0.0,0.429,4.167,1.305,...,2.581,1803.987,597.029,9.59,NaN,NaN,0.5,64.83,0.511,90
37,Afghanistan,2020-04-01,192.0,26.0,16.857,4.0,0.0,0.429,4.820,0.653,...,2.581,1803.987,597.029,9.59,NaN,NaN,0.5,64.83,0.511,91
38,Afghanistan,2020-04-02,235.0,43.0,22.143,4.0,0.0,0.286,5.899,1.079,...,2.581,1803.987,597.029,9.59,NaN,NaN,0.5,64.83,0.511,92
39,Afghanistan,2020-04-03,269.0,34.0,25.429,5.0,1.0,0.429,6.753,0.854,...,2.581,1803.987,597.029,9.59,NaN,NaN,0.5,64.83,0.511,93
40,Afghanistan,2020-04-04,270.0,1.0,23.429,5.0,0.0,0.429,6.778,0.025,...,2.581,1803.987,597.029,9.59,NaN,NaN,0.5,64.83,0.511,94
41,Afghanistan,2020-04-05,299.0,29.0,26.429,7.0,2.0,0.429,7.506,0.728,...,2.581,1803.987,597.029,9.59,NaN,NaN,0.5,64.83,0.511,95
42,Afghanistan,2020-04-06,337.0,38.0,31.857,7.0,0.0,0.429,8.460,0.954,...,2.581,1803.987,597.029,9.59,NaN,NaN,0.5,64.83,0.511,96


In [70]:
# Check the datatype of the new column to be an integer.
print(hum_dev_general_df.covid_days.dtype)

int64


In [71]:
# Swap the covid_days and date columns in a new DataFrame
days_general_df = hum_dev_general_df.copy()
cols = list(days_general_df.columns)
a, b = cols.index('date'), cols.index('covid_days')
cols[b], cols[a] = cols[a], cols[b]
days_general_df = days_general_df[cols].drop(columns=['date'], axis=1)
print(days_general_df.shape)
days_general_df.sample(5)

(122198, 32)


,location,covid_days,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,...,median_age,aged_65_older,gdp_per_capita,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,hospital_beds_per_thousand,life_expectancy,human_development_index
id_row,,,,,,,,,,,,,,,,,,,,,
89721,Italy,725,5647313.0,24882.0,36879.714,136611.0,81.0,138.571,93548.941,412.176,...,47.9,23.021,35220.084,113.151,4.78,19.8,27.8,3.18,83.51,0.892
50328,Dominican Republic,508,283729.0,1044.0,934.429,3606.0,0.0,3.429,25902.539,95.310,...,27.6,6.981,14600.861,266.653,8.20,8.5,19.1,1.60,74.08,0.756
137604,Pakistan,470,745182.0,5364.0,4907.571,15982.0,110.0,107.571,3308.980,23.819,...,23.5,4.495,5034.708,423.031,8.35,2.8,36.7,0.60,67.27,0.557
184494,Uganda,688,127128.0,55.0,42.143,3256.0,9.0,2.857,2697.760,1.167,...,16.4,2.168,1697.707,213.333,2.50,3.4,16.7,0.50,63.37,0.544
106379,Luxembourg,321,27681.0,425.0,567.286,240.0,4.0,6.000,43604.898,669.487,...,39.7,14.312,94277.965,128.275,4.42,20.9,26.0,4.51,82.25,0.916


In [72]:
days_general_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 122198 entries, 34 to 198747
Data columns (total 32 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   location                             122198 non-null  object 
 1   covid_days                           122198 non-null  int64  
 2   total_cases                          122198 non-null  float64
 3   new_cases                            122198 non-null  float64
 4   new_cases_smoothed                   122084 non-null  float64
 5   total_deaths                         120313 non-null  float64
 6   new_deaths                           120181 non-null  float64
 7   new_deaths_smoothed                  119718 non-null  float64
 8   total_cases_per_million              122198 non-null  float64
 9   new_cases_per_million                122114 non-null  float64
 10  new_cases_smoothed_per_million       122084 non-null  float64
 11  total_deaths

#### Create Daily Vaccinations

In [73]:
# Create a new column to calculate the daily difference in total vaccinations 
daily_vac_general_df = days_general_df.copy()
daily_vac_general_df['daily_vaccinations'] = daily_vac_general_df.groupby(['location'])['total_vaccinations'].diff()
daily_vac_general_df.head()

,location,covid_days,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,...,aged_65_older,gdp_per_capita,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,hospital_beds_per_thousand,life_expectancy,human_development_index,daily_vaccinations
id_row,,,,,,,,,,,,,,,,,,,,,
34,Afghanistan,88,114.0,8.0,11.429,4.0,2.0,0.571,2.862,0.201,...,2.581,1803.987,597.029,9.59,NaN,NaN,0.5,64.83,0.511,NaN
35,Afghanistan,89,114.0,0.0,10.571,4.0,0.0,0.429,2.862,0.000,...,2.581,1803.987,597.029,9.59,NaN,NaN,0.5,64.83,0.511,0.0
36,Afghanistan,90,166.0,52.0,17.714,4.0,0.0,0.429,4.167,1.305,...,2.581,1803.987,597.029,9.59,NaN,NaN,0.5,64.83,0.511,0.0
37,Afghanistan,91,192.0,26.0,16.857,4.0,0.0,0.429,4.820,0.653,...,2.581,1803.987,597.029,9.59,NaN,NaN,0.5,64.83,0.511,0.0
38,Afghanistan,92,235.0,43.0,22.143,4.0,0.0,0.286,5.899,1.079,...,2.581,1803.987,597.029,9.59,NaN,NaN,0.5,64.83,0.511,0.0


In [74]:
# Confirm calculations
check10_df = daily_vac_general_df[['location', 'total_vaccinations', 'daily_vaccinations']]
check10_df.head()

,location,total_vaccinations,daily_vaccinations
id_row,,,
34,Afghanistan,0.0,NaN
35,Afghanistan,0.0,0.0
36,Afghanistan,0.0,0.0
37,Afghanistan,0.0,0.0
38,Afghanistan,0.0,0.0


In [77]:
# Confirm daily_vaccines calculations are grouped by location. 
locations = ['Zambia','Zimbabwe']
zim = check10_df.loc[check10_df.location.isin(locations)]
zim.location.value_counts()

Zambia      697
Zimbabwe    670
Name: location, dtype: int64

In [79]:
zim.index[[zim.location.isin(['Zambia']) == True]].tolist()

[197214,
 197215,
 197216,
 197217,
 197218,
 197219,
 197220,
 197221,
 197222,
 197223,
 197224,
 197225,
 197226,
 197227,
 197228,
 197229,
 197230,
 197231,
 197232,
 197233,
 197234,
 197235,
 197236,
 197237,
 197238,
 197239,
 197240,
 197241,
 197242,
 197243,
 197244,
 197245,
 197246,
 197247,
 197248,
 197249,
 197250,
 197251,
 197252,
 197253,
 197254,
 197255,
 197256,
 197257,
 197258,
 197259,
 197260,
 197261,
 197262,
 197263,
 197264,
 197265,
 197266,
 197267,
 197268,
 197269,
 197270,
 197271,
 197272,
 197273,
 197274,
 197275,
 197276,
 197277,
 197278,
 197279,
 197280,
 197281,
 197282,
 197283,
 197284,
 197285,
 197286,
 197287,
 197288,
 197289,
 197290,
 197291,
 197292,
 197293,
 197294,
 197295,
 197296,
 197297,
 197298,
 197299,
 197300,
 197301,
 197302,
 197303,
 197304,
 197305,
 197306,
 197307,
 197308,
 197309,
 197310,
 197311,
 197312,
 197313,
 197314,
 197315,
 197316,
 197317,
 197318,
 197319,
 197320,
 197321,
 197322,
 197323,
 197324,
 

In [80]:
zim.index[[zim.location.isin(['Zimbabwe']) == True]].tolist()

[198078,
 198079,
 198080,
 198081,
 198082,
 198083,
 198084,
 198085,
 198086,
 198087,
 198088,
 198089,
 198090,
 198091,
 198092,
 198093,
 198094,
 198095,
 198096,
 198097,
 198098,
 198099,
 198100,
 198101,
 198102,
 198103,
 198104,
 198105,
 198106,
 198107,
 198108,
 198109,
 198110,
 198111,
 198112,
 198113,
 198114,
 198115,
 198116,
 198117,
 198118,
 198119,
 198120,
 198121,
 198122,
 198123,
 198124,
 198125,
 198126,
 198127,
 198128,
 198129,
 198130,
 198131,
 198132,
 198133,
 198134,
 198135,
 198136,
 198137,
 198138,
 198139,
 198140,
 198141,
 198142,
 198143,
 198144,
 198145,
 198146,
 198147,
 198148,
 198149,
 198150,
 198151,
 198152,
 198153,
 198154,
 198155,
 198156,
 198157,
 198158,
 198159,
 198160,
 198161,
 198162,
 198163,
 198164,
 198165,
 198166,
 198167,
 198168,
 198169,
 198170,
 198171,
 198172,
 198173,
 198174,
 198175,
 198176,
 198177,
 198178,
 198179,
 198180,
 198181,
 198182,
 198183,
 198184,
 198185,
 198186,
 198187,
 198188,
 

In [87]:
zim.loc[197897:198081]

,location,total_vaccinations,daily_vaccinations
id_row,,,
197897,Zambia,0.0,-3178009.0
197898,Zambia,3216653.0,3216653.0
197899,Zambia,0.0,-3216653.0
197900,Zambia,3237700.0,3237700.0
197901,Zambia,3267529.0,29829.0
197902,Zambia,0.0,-3267529.0
197903,Zambia,3288541.0,3288541.0
197904,Zambia,0.0,-3288541.0
197905,Zambia,3325582.0,3325582.0


In [85]:
# Count the number of negative daily_vaccinations
neg_vac = daily_vac_general_df.loc[daily_vac_general_df.daily_vaccinations < 0]
neg_vac.shape

(6348, 33)

In [88]:
# Drop negative difference in total_vaccination since it indicated gaps in reporting total_vacciantions.
nogap_daily_vac_general_df = daily_vac_general_df.copy()
nogap_daily_vac_general_df.drop(nogap_daily_vac_general_df[nogap_daily_vac_general_df.daily_vaccinations < 0].index, inplace=True)
nogap_daily_vac_general_df.shape

(115850, 33)

In [90]:
check11_df = nogap_daily_vac_general_df[['location', 'total_vaccinations', 'daily_vaccinations']]
zim1 = check11_df.loc[check11_df.location.isin(locations)]
zim1.loc[197897:198081]

,location,total_vaccinations,daily_vaccinations
id_row,,,
197898,Zambia,3216653.0,3216653.0
197900,Zambia,3237700.0,3237700.0
197901,Zambia,3267529.0,29829.0
197903,Zambia,3288541.0,3288541.0
197905,Zambia,3325582.0,3325582.0
197906,Zambia,3345769.0,20187.0
197908,Zambia,0.0,0.0
197909,Zambia,0.0,0.0
197910,Zambia,3390539.0,3390539.0


In [91]:
# Check the count of missing differences in daily vaccinations
nogap_daily_vac_general_df.daily_vaccinations.isnull().sum()

177

In [92]:
# Fill the null first records of difference calculations with zeros. 
nogap_daily_vac_general_df['daily_vaccinations'] = nogap_daily_vac_general_df['daily_vaccinations'].fillna(0)
print(nogap_daily_vac_general_df.shape)
nogap_daily_vac_general_df['daily_vaccinations'].isnull().sum()

(115850, 33)


0

#### Create Daily People Fully Vaccinated 

In [93]:
# Create a new column to calculate the daily difference in people_fully_vaccinated  
daily_ppl_vac_general_df = nogap_daily_vac_general_df.copy()
daily_ppl_vac_general_df['daily_people_fully_vaccinated'] = daily_ppl_vac_general_df.groupby(['location'])['people_fully_vaccinated'].diff()
daily_ppl_vac_general_df.head()

,location,covid_days,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,...,gdp_per_capita,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,hospital_beds_per_thousand,life_expectancy,human_development_index,daily_vaccinations,daily_people_fully_vaccinated
id_row,,,,,,,,,,,,,,,,,,,,,
34,Afghanistan,88,114.0,8.0,11.429,4.0,2.0,0.571,2.862,0.201,...,1803.987,597.029,9.59,NaN,NaN,0.5,64.83,0.511,0.0,NaN
35,Afghanistan,89,114.0,0.0,10.571,4.0,0.0,0.429,2.862,0.000,...,1803.987,597.029,9.59,NaN,NaN,0.5,64.83,0.511,0.0,0.0
36,Afghanistan,90,166.0,52.0,17.714,4.0,0.0,0.429,4.167,1.305,...,1803.987,597.029,9.59,NaN,NaN,0.5,64.83,0.511,0.0,0.0
37,Afghanistan,91,192.0,26.0,16.857,4.0,0.0,0.429,4.820,0.653,...,1803.987,597.029,9.59,NaN,NaN,0.5,64.83,0.511,0.0,0.0
38,Afghanistan,92,235.0,43.0,22.143,4.0,0.0,0.286,5.899,1.079,...,1803.987,597.029,9.59,NaN,NaN,0.5,64.83,0.511,0.0,0.0


In [95]:
# Confirm calculations
check12_df = daily_ppl_vac_general_df[['location', 'people_fully_vaccinated', 'daily_people_fully_vaccinated']]
check12_df.head()

,location,people_fully_vaccinated,daily_people_fully_vaccinated
id_row,,,
34,Afghanistan,0.0,NaN
35,Afghanistan,0.0,0.0
36,Afghanistan,0.0,0.0
37,Afghanistan,0.0,0.0
38,Afghanistan,0.0,0.0


In [96]:
# Confirm daily_people_fully_vaccinated calculations are grouped by location. 
locations = ['Zambia','Zimbabwe']
zim2 = check12_df.loc[check12_df.location.isin(locations)]
zim2.location.value_counts()

Zimbabwe    642
Zambia      623
Name: location, dtype: int64

In [97]:
zim2.loc[197897:198081]

,location,people_fully_vaccinated,daily_people_fully_vaccinated
id_row,,,
197898,Zambia,2082989.0,19613.0
197900,Zambia,2094075.0,11086.0
197901,Zambia,2109452.0,15377.0
197903,Zambia,2120529.0,11077.0
197905,Zambia,2140910.0,20381.0
197906,Zambia,2152077.0,11167.0
197908,Zambia,0.0,-2152077.0
197909,Zambia,0.0,0.0
197910,Zambia,2176230.0,2176230.0


In [98]:
# Count the number of negative daily_people_fully_vaccinated
neg_ppl = daily_ppl_vac_general_df.loc[daily_ppl_vac_general_df.daily_people_fully_vaccinated < 0]
neg_ppl.shape

(3781, 34)

In [99]:
# Drop negative difference in daily_people_fully_vaccinated since it indicated gaps in reporting people_fully_vaccinated.
nogap_daily_ppl_vac_general_df = daily_ppl_vac_general_df.copy()
nogap_daily_ppl_vac_general_df.drop(nogap_daily_ppl_vac_general_df[nogap_daily_ppl_vac_general_df.daily_people_fully_vaccinated < 0].index, inplace=True)
nogap_daily_ppl_vac_general_df.shape

(112069, 34)

In [100]:
check13_df = nogap_daily_ppl_vac_general_df[['location', 'people_fully_vaccinated', 'daily_people_fully_vaccinated']]
zim2 = check13_df.loc[check13_df.location.isin(locations)]
zim2.loc[197897:198081]

,location,people_fully_vaccinated,daily_people_fully_vaccinated
id_row,,,
197898,Zambia,2082989.0,19613.0
197900,Zambia,2094075.0,11086.0
197901,Zambia,2109452.0,15377.0
197903,Zambia,2120529.0,11077.0
197905,Zambia,2140910.0,20381.0
197906,Zambia,2152077.0,11167.0
197909,Zambia,0.0,0.0
197910,Zambia,2176230.0,2176230.0
198078,Zimbabwe,0.0,NaN


In [101]:
# Check the count of missing differences in daily people vaccinated
nogap_daily_ppl_vac_general_df.daily_people_fully_vaccinated.isnull().sum()

177

In [102]:
# Fill the null first records of difference calculations with zeros. 
nogap_daily_ppl_vac_general_df['daily_people_fully_vaccinated'] = nogap_daily_ppl_vac_general_df['daily_people_fully_vaccinated'].fillna(0)
print(nogap_daily_ppl_vac_general_df.shape)
nogap_daily_ppl_vac_general_df['daily_people_fully_vaccinated'].isnull().sum()

(112069, 34)


0

In [103]:
nogap_daily_ppl_vac_general_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112069 entries, 34 to 198747
Data columns (total 34 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   location                             112069 non-null  object 
 1   covid_days                           112069 non-null  int64  
 2   total_cases                          112069 non-null  float64
 3   new_cases                            112069 non-null  float64
 4   new_cases_smoothed                   111964 non-null  float64
 5   total_deaths                         110197 non-null  float64
 6   new_deaths                           110070 non-null  float64
 7   new_deaths_smoothed                  109612 non-null  float64
 8   total_cases_per_million              112069 non-null  float64
 9   new_cases_per_million                111991 non-null  float64
 10  new_cases_smoothed_per_million       111964 non-null  float64
 11  total_deaths

#### Normalize Variables Per 100,000 People

In [104]:
# Create a new column to normalize total_cases per 100,000 people, using the provided total_cases_per_million column.
total_cases_100k_general_df = nogap_daily_ppl_vac_general_df.copy()
total_cases_100k_general_df["total_cases_per_100K"] = total_cases_100k_general_df["total_cases_per_million"] / 10
print(total_cases_100k_general_df.shape)

(112069, 35)


In [106]:
check_df = total_cases_100k_general_df[["location", "total_cases", "population", "total_cases_per_million", "total_cases_per_100K"]]
check_df.sample(5)

,location,total_cases,population,total_cases_per_million,total_cases_per_100K
id_row,,,,,
101146,Liberia,1311.0,5180208.0,253.079,25.3079
114341,Mauritania,37094.0,4775110.0,7768.198,776.8198
7863,Armenia,13130.0,2968128.0,4423.664,442.3664
123088,Myanmar,113082.0,54806014.0,2063.314,206.3314
122363,Mozambique,67729.0,32163045.0,2105.802,210.5802


In [107]:
# Drop the older column 'total_cases_per_million'
total_cases_100k_general_df = total_cases_100k_general_df.drop(columns="total_cases_per_million", axis=1)
print(total_cases_100k_general_df.shape)

(112069, 34)


In [108]:
# Create a new column to normalize new_cases per 100,000 people, using the provided new_cases_per_million column.
new_cases_100k_general_df = total_cases_100k_general_df.copy()
new_cases_100k_general_df["new_cases_per_100K"] = new_cases_100k_general_df["new_cases_per_million"] / 10
print(new_cases_100k_general_df.shape)

(112069, 35)


In [109]:
check_1_df = new_cases_100k_general_df[["location", "new_cases", "population", "new_cases_per_million", "new_cases_per_100K"]]
check_1_df.sample(5)

,location,new_cases,population,new_cases_per_million,new_cases_per_100K
id_row,,,,,
142976,Philippines,1581.0,1.110469e+08,14.237,1.4237
186509,United Kingdom,612.0,6.820711e+07,8.973,0.8973
178541,Togo,17.0,8.478242e+06,2.005,0.2005
37180,China,26.0,1.444216e+09,0.018,0.0018
24899,Brazil,21807.0,2.139934e+08,101.905,10.1905


In [110]:
# Drop the older column 'new_cases_per_million'
new_cases_100k_general_df = new_cases_100k_general_df.drop(columns="new_cases_per_million", axis=1)
print(new_cases_100k_general_df.shape)

(112069, 34)


In [ ]:
# # Create a new column to normalize new_cases_smoothed per 100,000 people, using the provided new_cases_smoothed_per_million column.
# avg_new_cases_general_df = new_cases_general_df.copy()
# avg_new_cases_general_df["new_cases_smoothed_per_100K"] = avg_new_cases_general_df["new_cases_smoothed_per_million"] / 10
# print(avg_new_cases_general_df.shape)

In [ ]:
# check_2_df = avg_new_cases_general_df[["location", "date", "new_cases_smoothed", "population", "new_cases_smoothed_per_million", "new_cases_smoothed_per_100K"]]
# check_2_df.sample(5)

In [ ]:
# # Drop the older column 'new_cases_smoothed_per_million'
# avg_new_cases_general_df = avg_new_cases_general_df.drop(columns="new_cases_smoothed_per_million", axis=1)
# print(avg_new_cases_general_df.shape)

Normalize:
- **total_vaccinations_per_100K** = total_vaccinations_per_hundred * 1000
- **people_fully_vaccinated_per_100K** = people_fully_vaccinated_per_hundred * 1000

In [111]:
# Create a new column to normalize total_vaccinations_per_hundred
total_vac_100k_general_df = new_cases_100k_general_df.copy()
total_vac_100k_general_df["total_vaccinations_per_100K"] = total_vac_100k_general_df["total_vaccinations_per_hundred"] * 1000
print(total_vac_100k_general_df.shape)

(112069, 35)


In [112]:
check_3_df = total_vac_100k_general_df[["location", "total_vaccinations", "population", "total_vaccinations_per_hundred", "total_vaccinations_per_100K"]]
check_3_df.sample(5)

,location,total_vaccinations,population,total_vaccinations_per_hundred,total_vaccinations_per_100K
id_row,,,,,
18271,Belize,0.0,404915.0,0.00,0.0
76833,Guyana,0.0,790329.0,0.00,0.0
32210,Canada,54516547.0,38067913.0,143.21,143210.0
8314,Armenia,0.0,2968128.0,0.00,0.0
51444,Ecuador,30451413.0,17888474.0,170.23,170230.0


In [113]:
# Drop the older column 'total_vaccinations_per_hundred'
total_vac_100k_general_df = total_vac_100k_general_df.drop(columns=["total_vaccinations_per_hundred"], axis=1)
print(total_vac_100k_general_df.shape)

(112069, 34)


In [114]:
# Create a new column to normalize people_fully_vaccinated_per_hundred 
ppl_vac_100k_general_df = total_vac_100k_general_df.copy()
ppl_vac_100k_general_df["people_fully_vaccinated_per_100K"] = ppl_vac_100k_general_df["people_fully_vaccinated_per_hundred"] * 1000
print(ppl_vac_100k_general_df.shape)

(112069, 35)


In [115]:
check_4_df = ppl_vac_100k_general_df[["location", "people_fully_vaccinated", "population", "people_fully_vaccinated_per_hundred", "people_fully_vaccinated_per_100K"]]
check_4_df.sample(5)

,location,people_fully_vaccinated,population,people_fully_vaccinated_per_hundred,people_fully_vaccinated_per_100K
id_row,,,,,
194651,Vietnam,0.0,98168829.0,0.00,0.0
24534,Botswana,0.0,2397240.0,0.00,0.0
137429,Pakistan,0.0,225199929.0,0.00,0.0
136691,Oman,19019.0,5223376.0,0.36,360.0
42942,Croatia,2219989.0,4081657.0,54.39,54390.0


In [116]:
# Drop the older column 'people_fully_vaccinated_per_hundred'
ppl_vac_100k_general_df = ppl_vac_100k_general_df.drop(columns=["people_fully_vaccinated_per_hundred"], axis=1)
print(ppl_vac_100k_general_df.shape)

(112069, 34)


In [117]:
ppl_vac_100k_general_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112069 entries, 34 to 198747
Data columns (total 34 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   location                          112069 non-null  object 
 1   covid_days                        112069 non-null  int64  
 2   total_cases                       112069 non-null  float64
 3   new_cases                         112069 non-null  float64
 4   new_cases_smoothed                111964 non-null  float64
 5   total_deaths                      110197 non-null  float64
 6   new_deaths                        110070 non-null  float64
 7   new_deaths_smoothed               109612 non-null  float64
 8   new_cases_smoothed_per_million    111964 non-null  float64
 9   total_deaths_per_million          110197 non-null  float64
 10  new_deaths_per_million            110070 non-null  float64
 11  new_deaths_smoothed_per_million   109612 non-null  

In [124]:
# Create a new column to normalize daily_vaccinations per 100,000 people
daily_vac_100k_general_df = ppl_vac_100k_general_df.copy()

def daily_vac_100K(df_sub):
    daily_vac_100k_general_df['daily_vaccinations_per_100K'] = daily_vac_100k_general_df['daily_vaccinations'] / daily_vac_100k_general_df['population'] * 100000
    return df_sub

daily_vac_100k_general_df.groupby('location').apply(daily_vac_100K)
print(daily_vac_100k_general_df.shape)

(112069, 35)


In [125]:
daily_vac_100k_general_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112069 entries, 34 to 198747
Data columns (total 35 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   location                          112069 non-null  object 
 1   covid_days                        112069 non-null  int64  
 2   total_cases                       112069 non-null  float64
 3   new_cases                         112069 non-null  float64
 4   new_cases_smoothed                111964 non-null  float64
 5   total_deaths                      110197 non-null  float64
 6   new_deaths                        110070 non-null  float64
 7   new_deaths_smoothed               109612 non-null  float64
 8   new_cases_smoothed_per_million    111964 non-null  float64
 9   total_deaths_per_million          110197 non-null  float64
 10  new_deaths_per_million            110070 non-null  float64
 11  new_deaths_smoothed_per_million   109612 non-null  

In [129]:
check_5_df = daily_vac_100k_general_df[["location", "daily_vaccinations", "population", "daily_vaccinations_per_100K"]]
check_5_df.sample(5)

,location,daily_vaccinations,population,daily_vaccinations_per_100K
id_row,,,,
2278,Albania,1373349.0,2872934.0,47803.01253
25447,Brazil,609264.0,213993441.0,284.71153
94963,Kiribati,0.0,121388.0,0.00000
144559,Portugal,0.0,10167923.0,0.00000
178268,Timor,0.0,1343875.0,0.00000


In [130]:
# Create a new column to normalize daily_people_fully_vaccinated per 100,000 people
daily_ppl_100k_general_df = daily_vac_100k_general_df.copy()

def daily_ppl_vac_100K(df_sub):
    daily_ppl_100k_general_df['daily_people_fully_vaccinated_per_100K'] = daily_ppl_100k_general_df['daily_people_fully_vaccinated'] / daily_ppl_100k_general_df['population'] * 100000
    return df_sub

daily_ppl_100k_general_df.groupby('location').apply(daily_ppl_vac_100K)
print(daily_ppl_100k_general_df.shape)

(112069, 36)


In [133]:
check_6_df = daily_ppl_100k_general_df[["location", "daily_people_fully_vaccinated", "population", "daily_people_fully_vaccinated_per_100K"]]
check_6_df.sample(5)

,location,daily_people_fully_vaccinated,population,daily_people_fully_vaccinated_per_100K
id_row,,,,
27407,Bulgaria,0.0,6896655.0,0.000000
117592,Moldova,1867.0,4024025.0,46.396332
7366,Argentina,52012.0,45605823.0,114.046840
144225,Poland,4428.0,37797000.0,11.715215
91300,Japan,697197.0,126050796.0,553.107971


In [134]:
daily_ppl_100k_general_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112069 entries, 34 to 198747
Data columns (total 36 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   location                                112069 non-null  object 
 1   covid_days                              112069 non-null  int64  
 2   total_cases                             112069 non-null  float64
 3   new_cases                               112069 non-null  float64
 4   new_cases_smoothed                      111964 non-null  float64
 5   total_deaths                            110197 non-null  float64
 6   new_deaths                              110070 non-null  float64
 7   new_deaths_smoothed                     109612 non-null  float64
 8   new_cases_smoothed_per_million          111964 non-null  float64
 9   total_deaths_per_million                110197 non-null  float64
 10  new_deaths_per_million                  110

To predict the daily number of new COVID-19 cases, we will focus on processing the following columns and creating new features off of these: (total_cases, new_cases, total_cases_per_million, new_cases_per_million, reproduction_rate, total_vaccinations, people_fully_vaccinated, total_vaccinations_per_hundred, people_fully_vaccinated_per_hundred, stringency_index, population, population_density, median_age, aged_65_older, gdp_per_capita, cardiovasc_death_rate, diabetes_prevalence, life_expectancy, human_development_index).

### Split the dataset into 2 dataframe: one for cases prediction and another for deaths prediction.

In [136]:
# Create death_pred_df 
death_pred_df = daily_ppl_100k_general_df.copy()
print(death_pred_df.shape)

(112069, 36)


In [137]:
death_pred_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112069 entries, 34 to 198747
Data columns (total 36 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   location                                112069 non-null  object 
 1   covid_days                              112069 non-null  int64  
 2   total_cases                             112069 non-null  float64
 3   new_cases                               112069 non-null  float64
 4   new_cases_smoothed                      111964 non-null  float64
 5   total_deaths                            110197 non-null  float64
 6   new_deaths                              110070 non-null  float64
 7   new_deaths_smoothed                     109612 non-null  float64
 8   new_cases_smoothed_per_million          111964 non-null  float64
 9   total_deaths_per_million                110197 non-null  float64
 10  new_deaths_per_million                  110

In [138]:
# Export the deaths prediction DataFrame to a csv file.
death_output_file = Path("./Resources/death_pred.csv")
death_pred_df.to_csv(death_output_file, index=True)

In [141]:
#Creat cases_pred_df
cases_pred_df = daily_ppl_100k_general_df.copy()
outcome_col_drop = ["total_deaths", "new_deaths", "new_deaths_smoothed", "total_deaths_per_million",
            "new_deaths_per_million", "new_deaths_smoothed_per_million", "hospital_beds_per_thousand"]
cases_pred_df = cases_pred_df.drop(columns=outcome_col_drop, axis=1)
cases_pred_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112069 entries, 34 to 198747
Data columns (total 29 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   location                                112069 non-null  object 
 1   covid_days                              112069 non-null  int64  
 2   total_cases                             112069 non-null  float64
 3   new_cases                               112069 non-null  float64
 4   new_cases_smoothed                      111964 non-null  float64
 5   new_cases_smoothed_per_million          111964 non-null  float64
 6   reproduction_rate                       112069 non-null  float64
 7   total_vaccinations                      112069 non-null  float64
 8   people_fully_vaccinated                 112069 non-null  float64
 9   stringency_index                        112069 non-null  float64
 10  population                              112

**We left in the smoothed numbers and smokers columns for later decision in the ML code file.**

In [142]:
# Export the cases prediction DataFrame to a csv file.
cases_output_file = Path("./Resources/cases_pred.csv")
cases_pred_df.to_csv(cases_output_file, index=True)